In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/zihan/Desktop/ai-job-market-analyzer/notebooks/ds_salaries_cleaned.csv")
df.head()

,unnamed:_0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,UK,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,USA,L


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 607 entries, 0 to 606
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   unnamed:_0          607 non-null    int64 
 1   work_year           607 non-null    int64 
 2   experience_level    607 non-null    object
 3   employment_type     607 non-null    object
 4   job_title           607 non-null    object
 5   salary              607 non-null    int64 
 6   salary_currency     607 non-null    object
 7   salary_in_usd       607 non-null    int64 
 8   employee_residence  607 non-null    object
 9   remote_ratio        607 non-null    int64 
 10  company_location    607 non-null    object
 11  company_size        607 non-null    object
dtypes: int64(5), object(7)
memory usage: 57.0+ KB


In [4]:
df = df.drop(columns=["unnamed:_0"])


In [5]:
df.columns

Index(['work_year', 'experience_level', 'employment_type', 'job_title',
       'salary', 'salary_currency', 'salary_in_usd', 'employee_residence',
       'remote_ratio', 'company_location', 'company_size'],
      dtype='object')

In [6]:
TARGET = "salary_in_usd"

In [7]:
experience_map = {
    "EN": 0,
    "MI": 1,
    "SE": 2,
    "EX": 3
}

df["experience_level_ord"] = df["experience_level"].map(experience_map)


In [8]:
df[["experience_level", "experience_level_ord"]].drop_duplicates().sort_values("experience_level_ord")


,experience_level,experience_level_ord
5,EN,0
0,MI,1
1,SE,2
25,EX,3


In [9]:
df["employment_type_cat"] = df["employment_type"]
df["is_full_time"] = (df["employment_type"] == "FT").astype(int)


In [10]:
df[["employment_type", "employment_type_cat", "is_full_time"]].value_counts()


employment_type  employment_type_cat  is_full_time
FT               FT                   1               588
PT               PT                   0                10
CT               CT                   0                 5
FL               FL                   0                 4
Name: count, dtype: int64

In [11]:
df["fully_remote_flag"] = (df["remote_ratio"] == 100).astype(int)


In [12]:
df[["remote_ratio", "fully_remote_flag"]].value_counts().sort_index()


remote_ratio  fully_remote_flag
0             0                    127
50            0                     99
100           1                    381
Name: count, dtype: int64

In [13]:
def map_role(title: str) -> str:
    title = title.lower()

    if "data scientist" in title:
        return "Data Scientist"
    if "data engineer" in title or "big data" in title:
        return "Data Engineer"
    if "machine learning" in title or "ml engineer" in title:
        return "ML Engineer"
    if "data analyst" in title or "bi analyst" in title:
        return "Analyst"
    if "manager" in title:
        return "Manager"
    if "director" in title or "head of" in title:
        return "Director"
    if "research" in title:
        return "Research"
    if "architect" in title:
        return "Architect"
    else:
        return "Other"

df["role_category"] = df["job_title"].apply(map_role)
df["role_category"].value_counts()


role_category
Data Scientist    159
Data Engineer     159
Analyst           119
ML Engineer        68
Other              39
Manager            19
Research           17
Director           16
Architect          11
Name: count, dtype: int64